# Get all messages

In [ ]:
from main import get_channel_user_emoji_counts

df = get_channel_user_emoji_counts()
df

# Identify emoji type

In [ ]:
import pandas as pd

# Load in data/emojis-and-url.csv

custom_emoji_df = pd.read_csv('data/emojis-and-url.csv')
custom_emoji_df

# Get all of the custom emojis into a set

custom_emoji_set = set(custom_emoji_df['emoji_name'].values)
custom_emoji_set

# Create a new column emoji_type that is either 'custom' or 'official'

df['emoji_type'] = df['emoji_name'].apply(lambda x: 'custom' if x in custom_emoji_set else 'official')

# Rearrange columns to be channel, user, emoji_name, emoji_type, count

df = df[['channel', 'user', 'emoji_name', 'emoji_type', 'count']]
df

In [ ]:
# Get row where emoji_name is trollface

df[df['emoji_name'] == 'trollface']

## Resolve names

In [ ]:
import pandas as pd

# Read slack_ids_to_user_names.csv into a dataframe

user_id_to_username_df = pd.read_csv('data/slack_ids_to_user_names.csv')

# Convert the dataframe to a dictionary

user_id_to_username = user_id_to_username_df.set_index('user_id').to_dict()['user_name']
user_id_to_username

# Resolve the user ids to usernames via user_id_to_username

df['user'] = df.user.map(user_id_to_username)
df

# Drop Users who are NaN

In [ ]:
# Drop users who are NaN

df = df.dropna(subset=['user']).reset_index(drop=True)

df

# Eliminate all users for each emoji except the top user for each emoji

In [ ]:
kings = {}
for emoji, group in df.groupby('emoji_name'):
    inner_group = group.groupby('user').sum().sort_values('count', ascending=False)
    row = inner_group.iloc[0]
    count = row['count']

    # Get all rows with count == count
    rows = inner_group[inner_group['count'] == count]

    # Map kings[emoji] to a list of users with {'user': user, 'count': count}
    emoji_type = group['emoji_type'].iloc[0]
    top_users = [{**d, **{'emoji_type': emoji_type}} for d in rows.reset_index().to_dict(orient='records')]
    kings[emoji] = top_users

# Expand out kings into a bunch of (emoji, user, count) tuples

kings_tuples = [(emoji, user['emoji_type'], user['user'], user['count']) for emoji, users in kings.items() for user in users]

# Make kings into a dataframe with two columns: emoji_name and user

kings_df = pd.DataFrame.from_dict(kings_tuples)
kings_df.columns = ['emoji_name', 'emoji_type', 'user', 'count']
kings_df


# Add URLs

In [ ]:
from urls import add_urls

kings_df = add_urls(kings_df)
kings_df

# Add rendered emojis

In [ ]:
# Read in data/short_name_to_emoji.json

import json

import numpy as np

with open('data/short_name_to_emoji.json') as f:
    short_name_to_emoji = json.load(f)

# Map emoji_name to rendered_emoji via short_name_to_emoji and use emoji_name if it doesn't exist

kings_df['emoji'] = kings_df['emoji_name'].apply(lambda x: short_name_to_emoji.get(x, np.nan))
kings_df

# Make JSON

In [ ]:
kings_df = kings_df.applymap(lambda x: None if pd.isna(x) else x)

# Make kings_df a dict of user to {"emojis": [{"name": "emoji_name", "url": "emoji_url"}]}

kings_dict = {}
for user, group in kings_df.groupby('user'):
    emojis = []
    for row in group.itertuples():
        emojis.append({
            'name': row.emoji_name,
            'type': row.emoji_type,
            'url': row.emoji_url,
            'emoji': row.emoji,
            'count': row.count
        })
    kings_dict[user] = emojis

# Sort kings_dict by count

for user, emojis in kings_dict.items():
    emojis.sort(key=lambda x: x['count'], reverse=True)

kings_dict

In [ ]:
# Dump kings_dict to kings.json

import json

with open('data/kings.json', 'w') as f:
    json.dump(kings_dict, f, indent=2, default=str)

!head -n 50 data/kings.json

In [ ]:
!cp data/kings.json ../react/src/pages/UsersPage